In [1]:
import numpy as np

# patch for backward compatibility
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_
    
import gymnasium as gym

env = gym.make("Ant-v4", render_mode="human")  # replace with HalfCheetah-v4 or Walker2d-v4
obs = env.reset()

for _ in range(1000):
    env.render()
    action = env.action_space.sample()  # random action
    obs, reward, terminated, truncated, info = env.step(action)
    
    done = terminated or truncated
    if done:
        obs = env.reset()

env.close()

/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment Ant-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(
/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


In [2]:
print(env.observation_space)

Box(-inf, inf, (27,), float64)


In [3]:
print(env.action_space)

Box(-1.0, 1.0, (8,), float32)


In [10]:
import mujoco
import numpy as np
import mediapy as media
from scipy.spatial.transform import Rotation as R

# --- Load model and create simulation ---
model = mujoco.MjModel.from_xml_path("ant.xml")
data = mujoco.MjData(model)

# --- Gait parameters ---
t = 0.0
dt = model.opt.timestep
amplitude = 0.5
freq = 0.5
phase_offsets = np.array([0, np.pi, np.pi, 0, np.pi, 0, 0, np.pi])
# PID gains
Kp = 5.0  # proportional
Kd = 1.0  # derivative


# --- Video / renderer setup ---
height, width = 240, 320
n_seconds = 10
framerate = 30
n_frames = n_seconds * framerate
frames = []

# --- Offscreen rendering ---
with mujoco.Renderer(model, height, width) as renderer:
    for i in range(n_frames):
        # Step simulation to match video frame timing
        while data.time * framerate < i:
            # Get current torso angles and angular velocities
            # Get root quaternion (w, x, y, z)
            quat = data.qpos[3:7]

            # Convert to Euler angles (roll, pitch, yaw)
            r = R.from_quat([quat[1], quat[2], quat[3], quat[0]])  # Note: [x, y, z, w]
            roll, pitch, yaw = r.as_euler('xyz', degrees=False)
            roll_vel, pitch_vel, _ = data.qvel[3:6]

            # Compute stabilizing torques
            tau_roll  = -Kp*roll  - Kd*roll_vel
            tau_pitch = -Kp*pitch - Kd*pitch_vel

            # Apply to corresponding joints (replace with your actuator indices)
            # Assuming first two actuators control torso orientation (adjust if needed)
            data.ctrl[0] += tau_roll
            data.ctrl[1] += tau_pitch
            # Open-loop sine-wave control
            data.ctrl[2:] = amplitude * np.sin(2 * np.pi * freq * t + phase_offsets[2:])
            mujoco.mj_step(model, data)
            t += dt

        # Render current frame
        renderer.update_scene(data)  # use default camera
        frame = renderer.render()
        frames.append(frame)

# --- Play video ---
media.show_video(frames, fps=framerate)

This browser does not support the video tag.


In [5]:
env = gym.make("Ant-v4", render_mode="human")
obs, _ = env.reset()

for _ in range(1000):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)

    # Access mujoco data
    sim = env.unwrapped.data  
    model = env.unwrapped.model  

    # Body masses and COM positions
    masses = model.body_mass  # shape (num_bodies,)
    xpos = sim.xipos           # (num_bodies, 3), center of mass of each body in world frame

    # Full system CoM (mass-weighted average)
    com = np.sum(masses[:, None] * xpos, axis=0) / np.sum(masses)
    print("Center of Mass:", com)

    if terminated or truncated:
        obs, _ = env.reset()

env.close()

/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment Ant-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


Center of Mass: [0.06938763 0.04316904 0.8132135 ]
Center of Mass: [0.06545477 0.04840376 0.77560444]
Center of Mass: [0.06150969 0.05359995 0.71344967]
Center of Mass: [0.05771396 0.05875711 0.62670287]
Center of Mass: [0.02236133 0.08546952 0.559825  ]
Center of Mass: [-0.02244738  0.12057882  0.51478016]
Center of Mass: [-0.07926614  0.14748908  0.5479296 ]
Center of Mass: [-0.16149191  0.13910065  0.62904996]
Center of Mass: [-0.24339437  0.13033473  0.68616038]
Center of Mass: [-0.32529714  0.12155643  0.71871332]
Center of Mass: [-0.4072029   0.11274403  0.7267334 ]
Center of Mass: [-0.48351519  0.10389313  0.71757765]
Center of Mass: [-0.55327474  0.0959465   0.69520935]
Center of Mass: [-0.62313877  0.08779157  0.6484342 ]
Center of Mass: [-0.70240484  0.07226212  0.60307621]
Center of Mass: [-0.79462167  0.05065784  0.56297053]
Center of Mass: [-0.86299629  0.037191    0.53668585]
Center of Mass: [-0.9034542   0.03243321  0.5646559 ]
Center of Mass: [-0.93762597  0.03146378  0

/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


Center of Mass: [-0.31971562 -0.21250925  0.45112421]
Center of Mass: [-0.33586493 -0.2175355   0.46333568]
Center of Mass: [-0.35201074 -0.22255663  0.45100895]
Center of Mass: [-0.36344391 -0.22203208  0.42468328]
Center of Mass: [-0.36809674 -0.2343784   0.46356313]
Center of Mass: [-0.36708398 -0.25371234  0.54147038]
Center of Mass: [-0.36579655 -0.2731035   0.59484818]
Center of Mass: [-0.35976781 -0.28725094  0.630746  ]
Center of Mass: [-0.35368071 -0.30129039  0.64227815]
Center of Mass: [-0.34761677 -0.31533493  0.6292728 ]
Center of Mass: [-0.33856637 -0.32392169  0.60066793]
Center of Mass: [-0.31828421 -0.31084529  0.58592472]
Center of Mass: [-0.2950455  -0.28588912  0.5648015 ]
Center of Mass: [-0.27339748 -0.26487202  0.54130681]
Center of Mass: [-0.29542637 -0.27383938  0.59493015]
Center of Mass: [-0.31678665 -0.28274114  0.62307607]
Center of Mass: [-0.3380724  -0.29170063  0.62671472]
Center of Mass: [-0.36184282 -0.29648589  0.61519501]
Center of Mass: [-0.37831638